In [3]:
import tensorflow as tf
from tensorflow import keras

# Univariate CNN Models

In [6]:
from numpy import array

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    
    for i in range(len(sequence)):
        end_ix = i + n_steps
        
        if end_ix > len(sequence)-1:
            break
        
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    
    return array(X), array(y)

In [7]:
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90


In [10]:
# reshape data as 3D array
n_features = 1
X = X.reshape(X.shape[0], X.shape[1], n_features)

In [12]:
# define model
model = keras.models.Sequential([
    keras.layers.Conv1D(filters=64, kernel_size=2, activation="relu", input_shape=(n_steps, n_features)),
    keras.layers.MaxPooling1D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(1)
])

model.compile(optimizer="adam", loss="mse")

In [13]:
# fit the model
model.fit(X, y, epochs=2000, verbose=0)

In [16]:
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape(1, n_steps, n_features)
yhat = model.predict(x_input)
print(yhat)

[[101.4886]]


# Multivariate CNN Models

## Multiple Input Series

In [19]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

In [25]:
from numpy import hstack

# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape(len(in_seq1), 1)
in_seq2 = in_seq2.reshape(len(in_seq2), 1)
out_seq = out_seq.reshape(len(out_seq), 1)

# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
dataset

array([[ 10,  15,  25],
       [ 20,  25,  45],
       [ 30,  35,  65],
       [ 40,  45,  85],
       [ 50,  55, 105],
       [ 60,  65, 125],
       [ 70,  75, 145],
       [ 80,  85, 165],
       [ 90,  95, 185]])

In [29]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    
    for i in range(len(sequences)):
        end_ix = i + n_steps
        
        if end_ix > len(sequences):
            break
        
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    
    return array(X), array(y)

In [30]:
# choose a number of time steps
n_steps = 3

# convert into input/output
X, y = split_sequences(dataset, n_steps)
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

(7, 3, 2) (7,)
[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185


In [31]:
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]

In [32]:
# define model
model = keras.models.Sequential([
    keras.layers.Conv1D(filters=64, kernel_size=2, activation="relu", input_shape=(n_steps, n_features)),
    keras.layers.MaxPooling1D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(1)
])

model.compile(optimizer="adam", loss="mse")

In [33]:
# fit the model
model.fit(X, y, epochs=2000, verbose=0)

In [ ]:
# demonstrate prediction
x_input = array([[80, 85], [90, 95], [100, 105]])
x_input = x_input.reshape(1, n_steps, n_features)
yhat = model.predict(x_input)
print(yhat)

## Multiple Parallel Series

# Multi-step CNN Models

# Multivariate Multi-step CNN Models